### Using Naive Bayes Models To Predict Sleep Patterns Based On Average Daily Action Counts

### Imports

In [7]:
import modules
import numpy as np

In [8]:
X = modules.get_and_avg_data()
Y = X['depression_class'].copy()
X = X.drop(labels=['depression_class', 'depression_score'], axis = 1)
X = X.apply(modules.categorize_column, axis=0)

no user_tags: 520
no user_tags: 532
no user_tags: 503
no user_tags: 503
no user_tags: 523
no user_tags: 544
no user_tags: 529
no user_tags: 661
no user_tags: 658
no user_tags: 664
no user_tags: 634
no user_tags: 507
no user_tags: 547
no user_tags: 501
no user_tags: 668
no user_tags: 662
